In [54]:



# Step 2: Import libraries
import json
import os
import pinecone
import requests
import pandas as pd
import google.generativeai as genai
from pinecone import Pinecone, ServerlessSpec



In [59]:
data = json.load(open("reviews.json"))

In [23]:

# Step 4: Initialize Pinecone
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

# pc.create_index(
#   name="rag1", 
#   dimension=1536,
#   metric="cosine",
#   spec=ServerlessSpec(
#     cloud="aws",
#     region="us-east-1"
#   )
# )


In [24]:
index = pc.Index("rag1")

In [77]:
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

generation_config = {
    "temperature": 1,
    "top_p": 0.95,
    "top_k": 64,
    "max_output_tokens": 8192,
    "response_mime_type": "text/plain",
}

In [90]:
model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    generation_config=generation_config,
)

In [107]:
processed_data = []

# Assuming 'model' is properly initialized and refers to your Gemini model instance
print(model)

for review in data:
  
        # Generate the embedding using the Gemini model
        response = model.generate_content(
           review["review"]
        )
        embedding_text= response.candidates[0].content.parts['text']

        print(embedding_text)
        # Assume the embedding is returned as part of the response
        # Make sure to verify the structure of the response
        embedding = response['embedding']  # Use .get() for safety

        

        # Append processed data
        processed_data.append(
            {
                "values": embedding,
                "id": review["professor"],  # Use a unique identifier
                "metadata": {
                    "review": review["review"],
                    "subject": review["subject"],
                    "stars": review["stars"],
                }
            }
        )
 

# Now processed_data contains the embeddings and associated metadata


genai.GenerativeModel(
    model_name='models/gemini-1.5-flash',
    generation_config={'temperature': 1, 'top_p': 0.95, 'top_k': 64, 'max_output_tokens': 8192, 'response_mime_type': 'text/plain'},
    safety_settings={},
    tools=None,
    system_instruction=None,
    cached_content=None
)
[text: "This is a great start! It effectively describes Dr. Smith\'s teaching style and provides positive feedback. You could enhance it further by adding more specific details. Here are some ideas:\n\n**Adding specific details:**\n\n* **Examples of his teaching methods:** Does he use real-world examples, interactive activities, or visual aids to make concepts clear? \n* **Specific instances of clarity:**  Can you recall a particular lecture where Dr. Smith explained a challenging topic in a way that made it understandable? \n* **Impact on your learning:**  How has Dr. Smith\'s teaching influenced your understanding of the subject? \n\n**Adding a personal touch:**\n\n* **Why you appreciate his teac

TypeError: 'GenerateContentResponse' object is not subscriptable